# Project

Formula 1 is the highest class of single-seater auto racing sanctioned by the FIA, which inaugural season in 1950. The objective of the project was to dive into the rich history of Formula 1 racing, spanning over 70 years of data, to investigate whether certain steps of the race can be used to predict the winner of each Grand Prix. Through in-depth analysis of various race variables, the aime was to to uncover new insights and shed light on the fascinating world of high-stakes motorsport competition.

The dataset (obtained from __[Formula 1 World Championship (1950 - 2023)](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)__'s Kaggel Dataset) consists of all information on the Formula 1 races, drivers, constructors, qualifying, circuits, lap times, pit stops, championships from 1950 till the latest 2023 season.


Questions for analysis:

1. How accurately can the qualifying positions predict the race winners?
2. How accurately can the sprint positions predict the race winners?
3. Does the fastest lap of a GP indicates the winner?
4. Does pit stop strategy matters?

Findings summarized:
1. The race winner can be predicted from it's qualifying result on around 50%.
2. The race winner can be predicted from it's sprint result on around 33%.
3. The fastest lap can only indicate with an accuracy of 31% if the fastest drive will also be the winner.
4. The GP winner did on average 17,5% more pit stops than other drivers during the race.

# Analysis

In [1]:
#starting code

from pathlib import Path
import pandas as pd
import os
import sqlite3

directory = 'archive'
Path('F1_1950_2023.db').touch()

#CSV files are loaded into a database

conn = sqlite3.connect('F1_1950_2023.db')
c = conn.cursor()
for file in os.listdir(directory):
    if file.endswith(".csv"):
        filename = file.split('.csv')[0]
        df = pd.read_csv(f'{directory}/{file}')
        #print(filename,df)
        df.to_sql(f'{filename}', conn, if_exists='replace',
                  index=False)  # write the data to sqlite table
sql_query = """SELECT name FROM sqlite_master
WHERE type='table';"""
c.execute(sql_query)

## Qualifying

According to the code and findings bellow:

From the total of 448 races with qualifyings recorded in the dataset, in 227 races the winner was the same driver that obtained pole position (1st) during the qualifying stage.
Meaning that only around half of the races the first driver to qualify was the winner of the GP.

Therefore, how accurately can the qualifying positions predict the race winners?
Roughly in only 50% of the races.

Even taking into account the races since 2010 again only around 50% of the race winners could be predicted to be the same driver as the one in 1st position during the qualifying.

In [2]:
# How accurately can the qualifying positions predict the race winners?
c.execute('''
SELECT COUNT(*)
FROM qualifying, results, races
WHERE results.raceId=qualifying.raceId
    AND results.raceId=races.raceId
    AND results.position = 1 
    AND qualifying.position=1
    AND results.driverId <> qualifying.driverId
ORDER BY results.raceId ASC 
''')
qual = c.fetchall()
df = pd.DataFrame(qual)
df.style

,0
0,227


In [3]:
#since 2010
#total: 259
# <> : 129
#also around 50%

c.execute('''
SELECT COUNT(*)
FROM qualifying, results, races
WHERE results.raceId=qualifying.raceId
    AND results.raceId=races.raceId
    AND results.position = 1 
    AND qualifying.position=1
    AND year > 2009
    AND results.driverId <> qualifying.driverId
ORDER BY results.raceId ASC 
''')
qual = c.fetchall()
df = pd.DataFrame(qual)
df.style

,0
0,129


## Sprints

From the total of 6 sprints 4 of them had different winners, meaning that only 33% of the race winners could be predicted from the sprint winner.

In [4]:
#How accurately can the sprint positions predict the race winners?

c.execute('''
SELECT COUNT(*)
FROM sprint_results, results
WHERE results.raceId=sprint_results.raceId
    AND results.position = 1 
    AND sprint_results.position=1
    AND results.driverId <> sprint_results.driverId
ORDER BY results.raceId ASC 
''')
sprint = c.fetchall()
df = pd.DataFrame(sprint)
df.style

#total of sprints until today: 6
# <> 4
# 33% of who wins sprintes wins the race

,0
0,4


## Fastest lap

Only 31% of the race winner have the fastest lap on the race.

In [5]:
#Does the fastest lap of a GP indicates the winner?

c.execute('''
SELECT COUNT(*)
FROM results
WHERE position = 1 
    AND rank NOT LIKE '\%'
    AND position <> rank
ORDER BY raceId ASC
''')
flap = c.fetchall()
df = pd.DataFrame(flap)
df.style


# 366 race with time marked
# 252 races with the fastest lap not being the same driver as the race winner
# 31% with the fastest lap time are race winners

,0
0,252


## Pit stops

The pit stop analysis indicated that on average the winners of a GP do 17,5% more stops than other drivers during the race.

In [6]:
#Does pit stop strategy matters?

c.execute('''
SELECT results.raceId, results.driverId, stops.min, stops.max, stops.avg, driverstops.driverstops--, AVG(stops.avg)-AVG(driverstops.driverstops)
FROM (
  SELECT raceId, MIN(stop) AS min, MAX(stop) AS max, ROUND(AVG(stop), 1) AS avg
  FROM pit_stops 
  GROUP BY raceId 
  ORDER BY raceId
) AS stops
JOIN (
  SELECT raceId, driverId, COUNT(stop) AS driverstops
  FROM pit_stops
  GROUP BY driverId, raceId
  ORDER BY raceId
) AS driverstops ON stops.raceId = driverstops.raceId
JOIN results ON stops.raceId = results.raceId AND driverstops.driverId = results.driverId
WHERE results.position = 1
''')
qual = c.fetchall()
df = pd.DataFrame(qual)
df.style

#the winners are doing 17,5% more stops then the other drivers

,0,1,2,3,4,5
0,841,20,1,4,1.700000,2
1,842,20,1,4,2.000000,3
2,843,1,1,4,1.800000,3
3,844,20,1,4,2.300000,4
4,845,20,1,4,2.200000,4
5,846,20,1,3,1.600000,1
6,847,18,1,6,2.300000,6
7,848,20,1,3,1.900000,3
8,849,4,1,3,1.800000,3
9,850,1,1,3,1.700000,3
